# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
!python -m pip install semantic-kernel==0.3.15.dev0

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))


Let's define a prompt outlining a dialogue chat bot.

In [2]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [3]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

Initialize your context

In [4]:
context = kernel.create_new_context()
context["history"] = ""

Chat with the Bot

In [5]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?


Update the history with the output

In [6]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?



Keep Chatting!

In [7]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [8]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Sure! I would recommend "The Greeks: History, Culture, and Society" by Ian Morris and Barry B. Powell. It provides a comprehensive overview of ancient Greek history, culture, and society, covering topics such as politics, philosophy, art, and literature. It's a great book to learn more about Greece and its influence on Western civilization.


In [10]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Greeks: History, Culture, and Society" explores the history of ancient Greece from the Bronze Age to the Hellenistic period. It delves into various aspects of Greek civilization, including politics, religion, art, literature, philosophy, and daily life. The book examines the development of Greek city-states, the rise and fall of Athens, the Peloponnesian War, the conquests of Alexander the Great, and the spread of Hellenistic culture. It also discusses the contributions of famous Greek philosophers, such as Socrates, Plato, and Aristotle. Overall, it provides a comprehensive and insightful look into the rich history and culture of Greece.


In [12]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read "The Greeks: History, Culture, and Society," you will learn about the major events and developments in Greek history, such as the rise of the city-states, the Persian Wars, the Golden Age of Athens, the Peloponnesian War, and the conquests of Alexander the Great. You will also gain an understanding of Greek society and culture, including their political systems, religious beliefs, artistic achievements, and literary works. Additionally, the book explores the contributions of Greek philosophers and their impact on Western thought. Overall, it provides a comprehensive overview of Greek history and its significance in shaping Western civilization.


In [14]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are a few more books you might find interesting:

1. "The Peloponnesian War" by Thucydides - This ancient Greek historian's account of the war between Athens and Sparta provides valuable insights into the politics and military strategies of the time.

2. "The Iliad" by Homer - This epic poem tells the story of the Trojan War and offers a glimpse into ancient Greek mythology and heroism.

3. "The Republic" by Plato - In this philosophical work, Plato explores the nature of justice and the ideal society, presenting his famous allegory of the cave.

4. "The Histories" by Herodotus - Often considered the first work of history, this book covers a wide range of topics, including the Persian Wars and various cultures of the ancient world.

5. "The Odyssey" by Homer - This epic poem follows the adventures of Odysseus as he tries to return home after the Trojan War, providing insights into ancient Greek

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [15]:
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Sure! I would recommend "The Greeks: History, Culture, and Society" by Ian Morris and Barry B. Powell. It provides a comprehensive overview of ancient Greek history, culture, and society, covering topics such as politics, philosophy, art, and literature. It's a great book to learn more about Greece and its influence on Western civilization.

User: that sounds interesting, what is it about?
ChatBot: "The Greeks: History, Culture, and Society" explores the history of ancient Greece from the Bronze Age to the Hellenistic period. It delves into various aspects of Greek civilization, including politics, religion, art, literature, philosophy, and daily life. The book examines the development of Greek city-states, the rise and fall 